# EDT-Julia

EDT in Jualia language


In [10]:
import Pkg
Pkg.add("Images")
Pkg.add("Plots")

   Resolving package versions...
  No Changes to `C:\Users\masah\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\masah\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\masah\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\masah\.julia\environments\v1.7\Manifest.toml`


In [11]:
using Plots
using Images

In [33]:
function horizontal_edt(map)
    max_val = max(size(map,1), size(map,2))
    ret = (map .!= 0) * max_val

    function scan(x, y, min_val)
        f = ret[y,x]
        next = min_val + 1.
        v = min(f, next)
#         println(x, " ", y, " ", v, " ", sum(ret))
        ret[y,x] = v
        return v
    end

    for y = 1:size(map,1)
        min_val = 0
        for x = 1:size(map,2)
            min_val = scan(x, y, min_val)
        end
        for x = size(map,2):-1:1
            min_val = scan(x, y, min_val)
        end
    end
    return ret
end

horizontal_edt (generic function with 2 methods)

In [13]:
function edt(map)
    horz_edt = horizontal_edt(map)
    println("horz_edt: ", size(horz_edt))
    ret = zeros(size(map))

    function vertical_scan(x, y)
        column = ((1:size(map,1)) .- y) .^ 2 .+ horz_edt[:,x] .^ 2
        return minimum(column)
    end

    for x in 1:size(map,2)
        for y in 1:size(map,1)
            ret[y,x] = vertical_scan(x, y)
        end
    end
    
    return ret
end

edt (generic function with 1 method)

In [ ]:
SIZE = 512
x_ticks = range(-2, stop=2, length=SIZE)
y_ticks = range(-2, stop=2, length=SIZE)
X = [ j for i=x_ticks, j=y_ticks ]
Y = [ i for i=x_ticks, j=y_ticks ]
circle = convert(Array{Float64, 2}, X .^ 2 .+ Y .^ 2 .< 1.5 .^ 2)
# result = horizontal_edt(circle)
@time begin
result = edt(circle)
end
heatmap(result)

In [21]:
save("julia-edt.png", colorview(Gray, map(clamp01nan, result / maximum(result))))

In [41]:
rust_logo = load("Rust_logo.png")
typeof(rust_logo)
gray_rust_logo = Gray.(rust_logo)
typeof(gray_rust_logo)

Matrix{Gray{N0f8}} (alias for Array{Gray{Normed{UInt8, 8}}, 2})

In [ ]:
@time begin
result = edt(.~gray_rust_logo)
end
heatmap(result)